# Backtest sàn HOSE

In [2]:
#Nhập thư viện
from vnstock import *
import pandas as pd
import pandas_ta as ta
from backtesting import Strategy
from backtesting import Backtest
from backtesting.lib import crossover
import warnings
#Chặn warnings sharp ratio cho những cổ phiếu có returns = 0%
warnings.filterwarnings("ignore", category=RuntimeWarning)

#Lọc cổ phiếu
list = []
listhold = []
params = {
    "exchangeName": "HOSE",
    "avgTradingValue5Day":(100,500000),
    "marketCap": (100,500000),

}
data = stock_screening_insights(params, size=50)
#Set up và backtest hàng loạt
for i in data['ticker']:
    df= stock_historical_data(symbol=i, start_date="2024-03-01", end_date="2024-04-12", resolution="1D", type="stock", beautify=True, decor=False, source='DNSE')
    df['time']=pd.to_datetime(df['time'],format="%m/%d/%Y")
    df=df[df.high!=df.low]
    df.set_index("time", inplace=True)
    df["EMA_slow"]=ta.ema(df.close, length=12)
    df["EMA_fast"]=ta.ema(df.close, length=5)
    my_bbands = ta.bbands(df.close, length=7, std=1.2)
    df=df.join(my_bbands)
    pd.options.mode.chained_assignment = None
    def ema_signal(df, current_candle, backcandles):
        df_slice = df.reset_index().copy()
        # Rà hết vùng nến từ đầu đến cuối
        start = max(0, current_candle - backcandles)
        end = current_candle
        relevant_rows = df_slice.iloc[start:end]
    
        # Nếu tất cả EMA_fast ớ dưới EMA_slow, đề xuất lệnh bán
        if all(relevant_rows["EMA_fast"] < relevant_rows["EMA_slow"]):
            return 1  # bán
        # Nếu tất cả EMA_fast ớ dưới EMA_slow, đề xuất lệnh bán
        elif all(relevant_rows["EMA_fast"] > relevant_rows["EMA_slow"]):
            return 2  # mua
        else:
            return 0
    
    #df=df[-10000:-1]
    df.reset_index(inplace=True)
    df['EMASignal'] = df.apply(lambda row: ema_signal(df, row.name, 2) , axis=1)
    def bol_signal(df, current_candle):
        if df.close[current_candle]>=df['BBU_7_1.2'][current_candle]:
            return 1 #bán
        elif df.close[current_candle]<=df['BBL_7_1.2'][current_candle]:
            return 2 #mua
    df['Bol_Signal'] = df.apply(lambda row: bol_signal(df, row.name), axis=1)
    df.index=pd.to_datetime(df['time'],format='%Y/%m/%d')
    df = df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'})
    class MyStrat(Strategy):
    
        def init(self):
            pass
        def next(self):
            if self.data.EMASignal == 2 or self.data.Bol_Signal == 2:
                if len(self.trades)== 0:
                    self.buy()
            if self.data.EMASignal== 1 or self.data.Bol_Signal == 1:
                self.position.close()
    bt = Backtest(df, MyStrat, cash=10000000)
    run = bt.run()
    # print(i,f'Lợi nhuận: {round(run['Return [%]'],2)}%')#,run['Buy & Hold Return [%]'])
    # list.append(run['Return [%]'])
    #listhold.append(run['Buy & Hold Return [%]'])
# print(f'Tổng lợi nhuận: {sum(list)}%')
#print(sum(listhold))


ImportError: cannot import name 'stpipock_screening_insights' from 'vnstock' (d:\Anacoda\Lib\site-packages\vnstock\__init__.py)

# Backtest sàn HNX

In [22]:
# Nhập thư viện
from vnstock import *
import pandas as pd
import pandas_ta as ta
from backtesting import Strategy, Backtest
from backtesting.lib import crossover
import warnings
from vnstock import stock_screening_insights

# Chặn warnings sharp ratio cho những cổ phiếu có returns = 0%
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Lọc cổ phiếu
list = []
params = {
    "exchangeName": "HNX",
    "avgTradingValue5Day": (10, 5000),
    "marketCap": (100, 500000),
}

data = stock_screening_insights(params, size=50)

# Set up và backtest hàng loạt
for i in data['ticker']:
    df = stock_historical_data(symbol=i, start_date="2024-03-01", end_date="2024-09-12", resolution="1D", type="stock", beautify=True, decor=False, source='DNSE')
    df['time'] = pd.to_datetime(df['time'], format="%m/%d/%Y")
    df = df[df.high != df.low]
    df.set_index("time", inplace=True)

    # df = df.dropna()
    # Tính toán EMA, Bollinger Bands, MACD và RSI
    df["EMA_12"] = ta.ema(df.close, length=12)
    df["EMA_26"] = ta.ema(df.close, length=26)
    df["MACD"], df["MACD_signal"], df["MACD_diff"] = ta.macd(df.close, fast=12, slow=26, signal=9)
    df["RSI"] = ta.rsi(df.close, length=14)

    # Chuyển đổi cột MACD_diff về dạng float (số) nếu có dữ liệu kiểu chuỗi
    df['MACD_diff'] = pd.to_numeric(df['MACD_diff'], errors='coerce')

# Áp dụng tín hiệu MACD
    
    # Kiểm tra và xử lý các giá trị NaN hoặc None trước khi tính toán tín hiệu
      
    df = df.fillna(0, inplace = True)

    my_bbands = ta.bbands(df.close, length=14, std=1.5)
    df = df.join(my_bbands)
    
    # Tắt cảnh báo chained_assignment
    pd.options.mode.chained_assignment = None

    # Hàm xác định tín hiệu EMA
    def ema_signal(df, current_candle, backcandles):
        df_slice = df.reset_index().copy()
        start = max(0, current_candle - backcandles)
        end = current_candle
        relevant_rows = df_slice.iloc[start:end]

        # Kiểm tra điều kiện để mua/bán dựa trên EMA
        if all(relevant_rows["EMA_12"] < relevant_rows["EMA_26"]):
            return 1  # bán
        elif all(relevant_rows["EMA_12"] > relevant_rows["EMA_26"]):
            return 2  # mua
        else:
            return 0
    
    # Hàm xác định tín hiệu Bollinger Bands
    def bol_signal(df, current_candle):
        if df.close.iloc[current_candle] >= df['BBU_14_1.5'].iloc[current_candle]:
            return 1  # bán
        elif df.close.iloc[current_candle] <= df['BBL_14_1.5'].iloc[current_candle]:
            return 2  # mua
        else:
            return 0

    # Hàm xác định tín hiệu MACD
    def macd_signal(df, current_candle):
        if df["MACD_diff"].iloc[current_candle] > 0 and df["MACD_diff"].iloc[current_candle - 1] <= 0:
            return 2  # mua khi MACD cắt đường tín hiệu từ dưới lên
        elif df["MACD_diff"].iloc[current_candle] < 0 and df["MACD_diff"].iloc[current_candle - 1] >= 0:
            return 1  # bán khi MACD cắt đường tín hiệu từ trên xuống
        else:
            return 0

    # Hàm xác định tín hiệu RSI
    def rsi_signal(df, current_candle):
        if df["RSI"].iloc[current_candle] > 70:
            return 1  # bán khi RSI trên 70 (mua quá mức)
        elif df["RSI"].iloc[current_candle] < 30:
            return 2  # mua khi RSI dưới 30 (bán quá mức)
        else:
            return 0
    
    # Áp dụng tín hiệu vào DataFrame
    df.reset_index(inplace=True)
    df['EMASignal'] = df.apply(lambda row: ema_signal(df, row.name, 2), axis=1)
    df['Bol_Signal'] = df.apply(lambda row: bol_signal(df, row.name), axis=1)
    df['MACD_Signal'] = df.apply(lambda row: macd_signal(df, row.name), axis=1)
    df['RSI_Signal'] = df.apply(lambda row: rsi_signal(df, row.name), axis=1)

    # Định dạng lại cột cho phù hợp với Backtesting
    df.index = pd.to_datetime(df['time'], format='%Y/%m/%d')
    df = df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'})

    # Định nghĩa chiến lược
    class MyStrat(Strategy):
        def init(self):
            pass

        def next(self):
            # Kiểm tra tín hiệu mua từ EMA, Bollinger Bands, MACD, hoặc RSI
            if (self.data.EMASignal == 2 or self.data.Bol_Signal == 2 or 
                self.data.MACD_Signal == 2 or self.data.RSI_Signal == 2):  # Mua
                if len(self.trades) == 0:
                    self.buy()
            
            # Kiểm tra tín hiệu bán từ EMA, Bollinger Bands, MACD, hoặc RSI
            if (self.data.EMASignal == 1 or self.data.Bol_Signal == 1 or 
                self.data.MACD_Signal == 1 or self.data.RSI_Signal == 1):  # Bán
                if self.position.is_long:
                    self.position.close()

    # Thực hiện Backtest
    bt = Backtest(df, MyStrat, cash=10000000)
    run = bt.run()

    # In kết quả
    print(i, f'Lợi nhuận: {round(run["Return [%]"], 2)}%')
    list.append(run['Return [%]'])

# Tổng kết lợi nhuận
print(f'Tổng lợi nhuận: {sum(list)}%')


AttributeError: 'NoneType' object has no attribute 'close'

# Backtest sàn UPCOM

In [24]:
#Nhập thư viện
from vnstock import *
import pandas as pd
import pandas_ta as ta
from backtesting import Strategy
from backtesting import Backtest
from backtesting.lib import crossover
import warnings
from vnstock import stock_screening_insights
#Chặn warnings sharp ratio cho những cổ phiếu có returns = 0%
warnings.filterwarnings("ignore", category=RuntimeWarning)

#Lọc cổ phiếu
list = []
listhold = []
params = {
    "exchangeName": "UPCOM",
    "avgTradingValue5Day":(5,1000),
    "marketCap": (100,500000),
}
data = stock_screening_insights(params, size=50)
#Set up và backtest hàng loạt
for i in data['ticker']:
    df= stock_historical_data(symbol=i, start_date="2024-03-01", end_date="2024-04-12", resolution="1D", type="stock", beautify=True, decor=False, source='DNSE')
    df['time']=pd.to_datetime(df['time'],format="%m/%d/%Y")
    df=df[df.high!=df.low]
    df.set_index("time", inplace=True)
    df["EMA_slow"]=ta.ema(df.close, length=12)
    df["EMA_fast"]=ta.ema(df.close, length=5)
    my_bbands = ta.bbands(df.close, length=7, std=1.2)
    df=df.join(my_bbands)
    pd.options.mode.chained_assignment = None
    def ema_signal(df, current_candle, backcandles):
        df_slice = df.reset_index().copy()
        # Rà hết vùng nến từ đầu đến cuối
        start = max(0, current_candle - backcandles)
        end = current_candle
        relevant_rows = df_slice.iloc[start:end]
    
        # Nếu tất cả EMA_fast ớ dưới EMA_slow, đề xuất lệnh bán
        if all(relevant_rows["EMA_fast"] < relevant_rows["EMA_slow"]):
            return 1  # bán
        # Nếu tất cả EMA_fast ớ dưới EMA_slow, đề xuất lệnh bán
        elif all(relevant_rows["EMA_fast"] > relevant_rows["EMA_slow"]):
            return 2  # mua
        else:
            return 0
    
    #df=df[-10000:-1]
    df.reset_index(inplace=True)
    df['EMASignal'] = df.apply(lambda row: ema_signal(df, row.name, 2) , axis=1)
    def bol_signal(df, current_candle):
        if df.close[current_candle]>=df['BBU_7_1.2'][current_candle]:
            return 1 #bán
        elif df.close[current_candle]<=df['BBL_7_1.2'][current_candle]:
            return 2 #mua
    df['Bol_Signal'] = df.apply(lambda row: bol_signal(df, row.name), axis=1)
    df.index=pd.to_datetime(df['time'],format='%Y/%m/%d')
    df = df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'})
    class MyStrat(Strategy):
    
        def init(self):
            pass
        def next(self):
            if self.data.EMASignal == 2 or self.data.Bol_Signal == 2:
                if len(self.trades)== 0:
                    self.buy()
            if self.data.EMASignal== 1 or self.data.Bol_Signal == 1:
                self.position.close()
    bt = Backtest(df, MyStrat, cash=10000000)
    run = bt.run()
    print(i,f'Lợi nhuận: {round(run['Return [%]'],2)}%')#,run['Buy & Hold Return [%]'])
    list.append(run['Return [%]'])
    #listhold.append(run['Buy & Hold Return [%]'])
print(f'Tổng lợi nhuận: {sum(list)}%')
#print(sum(listhold))

AAS Lợi nhuận: -7.36%
ABB Lợi nhuận: -4.94%
ACV Lợi nhuận: -1.42%
BSR Lợi nhuận: 2.57%
BVB Lợi nhuận: -0.9%
C4G Lợi nhuận: -3.48%
DDV Lợi nhuận: 6.13%
DRI Lợi nhuận: -2.46%
MCH Lợi nhuận: -0.41%
NAB Lợi nhuận: 0.64%
PVP Lợi nhuận: 7.75%
QNS Lợi nhuận: 4.84%
SBS Lợi nhuận: -2.56%
SIP Lợi nhuận: -8.45%
VAB Lợi nhuận: 2.44%
VEA Lợi nhuận: 0.55%
VFS Lợi nhuận: 0.59%
VGI Lợi nhuận: 6.09%
VTP Lợi nhuận: -2.1%
Tổng lợi nhuận: -2.468%
